In [874]:
words = open('makemore/names.txt', 'r').read().splitlines()

In [875]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [876]:
import torch
N = torch.zeros(27, 27, dtype=torch.int32)
chs = sorted(list(set(''.join(words))))
b = dict(zip(chs, range(1, 27)))
b['.'] = 0

for w in words:
    w = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(w, w[1:]):
        idx1 = b[ch1]
        idx2 = b[ch2]
        N[idx1, idx2] += 1
b_reverse = {i: c for c, i in b.items()}
print(N[26])

tensor([160, 860,   4,   2,   2, 373,   0,   1,  43, 364,   2,   2, 123,  35,
          4, 110,   2,   0,  32,   4,   4,  73,   2,   3,   1, 147,  45],
       dtype=torch.int32)


In [877]:
P = N.float()
P = P / P.sum(1, keepdim=True)

In [878]:
g = torch.Generator().manual_seed(2147483647)
idx = 0
names = []
for _ in range(5):
    name = ""
    while True:
        d = P[idx]
        idx = torch.multinomial(d, 1, replacement=True, generator=g).item()
        ch = b_reverse[idx]
        name += ch
        if ch == '.':
            break
    names.append(name)
names

['junide.', 'janasah.', 'p.', 'cony.', 'a.']

In [912]:
# trian set
xs, ys = [], []

for w in words:
    w = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(w, w[1:]):
        idx1 = b[ch1]
        idx2 = b[ch2]
        xs.append(idx1)
        ys.append(idx2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(f"{num=}")

num=228146


In [913]:
import torch.nn.functional as F
W = torch.randn(27, 27).requires_grad_(True)

In [914]:
xenc = F.one_hot(xs, num_classes=27).float()

# train
for epoch in range(1000):
    logits = xenc @ W
    counts = logits.exp()
    prob = counts / counts.sum(1, keepdim=True)
    loss = -prob[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(f"{loss.item()=}")
    W.grad = None
    loss.backward()
    W.data += -50 * W.grad
"""for epoch in range(1000):
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(f"{loss.item()=}")
    W.grad = None # set to zero the gradient
    loss.backward()
    W.data += -50 * W.grad"""






loss.item()=3.729912042617798
loss.item()=3.3864452838897705
loss.item()=3.161362648010254
loss.item()=3.0093576908111572
loss.item()=2.9102420806884766
loss.item()=2.841439962387085
loss.item()=2.7913761138916016
loss.item()=2.753587245941162
loss.item()=2.7238717079162598
loss.item()=2.69966459274292
loss.item()=2.6794395446777344
loss.item()=2.6622731685638428
loss.item()=2.6475560665130615
loss.item()=2.634836435317993
loss.item()=2.623753786087036
loss.item()=2.6140170097351074
loss.item()=2.6053900718688965
loss.item()=2.597686290740967
loss.item()=2.590758800506592
loss.item()=2.5844905376434326
loss.item()=2.578789234161377
loss.item()=2.573578119277954
loss.item()=2.5687966346740723
loss.item()=2.5643930435180664
loss.item()=2.5603249073028564
loss.item()=2.5565555095672607
loss.item()=2.553053617477417
loss.item()=2.5497922897338867
loss.item()=2.5467488765716553
loss.item()=2.543903112411499
loss.item()=2.5412368774414062
loss.item()=2.5387356281280518
loss.item()=2.53638386

'for epoch in range(1000):\n    logits = xenc @ W # predict log-counts\n    counts = logits.exp() # counts, equivalent to N\n    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character\n    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()\n    print(f"{loss.item()=}")\n    W.grad = None # set to zero the gradient\n    loss.backward()\n    W.data += -50 * W.grad'

In [944]:
for _ in range(20):
    
    idx = 0
    output = []
    while True:
        xenc = F.one_hot(torch.tensor([idx]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdim=True)
        idx = torch.multinomial(probs, num_samples=1, replacement=True).item()
        output.append(b_reverse[idx])
        if idx == 0:
            break
    
    print(''.join(output))

n.
ve.
sanoiurila.
sy.
rmalifvieetilaah.
n.
dell.
rmuze.
n.
trden.
jelayabr.
dicon.
kiuradesan.
lyddigrle.
rliahi.
ja.
a.
iay.
fenaryad.
ima.


In [959]:
# W.data[:, 2] means features of the 2nd dimension
print(W.data[:, 2])

tensor([ 0.6145, -0.1469,  0.2262, -1.4266, -1.6661, -0.9058, -0.8977, -1.1297,
        -1.3089, -1.0524, -0.9421, -1.3620, -0.5937,  0.7396, -2.0247, -0.0111,
        -0.6831, -0.3351, -1.2156, -1.2311, -1.6848,  0.4175, -0.9412, -0.8530,
        -0.6872, -1.1880, -1.0301])


W: a matrix 27 by 27 <br/>
xenc: input `nums` by 27 <br/>
output: a matrix `nums` by 27 <br/><br/>
Understanding of xenc size: The latter `27` means input features in a certain space eg.embedding space and vetor space.<br/>
Understanding of W size: The `previous 27` means the number of features xenc has. And the `latter 27` unclose representations of the features of x hasing in the space where W is. It's difficult to understand probably. <br/>
Give an example for understanding easily.For example, we have 4 layers like input `2 × 16`, hidden_1 `16 × 32`, hidden_2 `32 × 8`, output `8 × 1`. Let me tell you every size meanings.Input layer 2 × 16 means we have 2 samples and features each of them are represented by a 16×1 vector(It's not caustions but it's meaning). The hidden_1 means input have 16 features and those features are represented by a 32 × 1 vector in hidden_1 layer. Like previous, hidden_2 means hidden_1 have 32 features and each of them is represented by a 8 × 1 vector in hidden_2 layer. And the last output layer means hidden_2 layer have 8 features and they are represented by a 1 × 1 vector in output layer. So during data processed through input -> hidden_1 and hidden_1 -> hidden_2, **it is experiencing higher dimensional and lower dimensional.** And during hidden_2 -> output, data is back to what we want it looks like. Maybe we want to data is represented by a 2×1 vector and then we make output size 8 × 2.<br/>
Here is a question that the benefit of data entering higer dimensional and lower dimensional. My intuition tells me that data have more features in a higher dimensional and less infomation in the lower dimensional. When data is processed in the higher dimensional, we can get more information to deal with data more correctly. When data downs from high dimenstion to low dimenstion, it is compressed. We often encounter some situations that there is few infomation to predict or solve the puzzle. Oh, right! Like a puzzle. When you play something games like puzzle, if you have more information of pieces the probability of you make it successfully is larger. Assume that one information means 1 feature, eg.we have 5 info that we can represent them by a 5 × 1 vector. Finally whatever information you have you have to solve the problem at 3-dimension. In other words, 5 × 1 vector transforms a representation in 3-dimenstion finally such as what output layer does.
<br/>  
Understanding of utput size: `nums` represent the previous character and the later `27` in W represents the number of next possible character given the previous character. So output[x, y] means the probabily of the next character `y` represents given character `x` represents.<br/>
Ok. It's all my understanding of size of network layers.